In [1]:
import pandas as pd
import numpy as np
from jupyter_dash import JupyterDash
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import dash_bootstrap_components as dbc

In [2]:
#external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
external_stylesheets = [dbc.themes.CERULEAN]

In [3]:
df_all= pd.read_csv('for_dash.csv')
df_all.drop('Unnamed: 0', axis=1, inplace=True)

In [4]:
df_all.head()

,amount,timestamp,day,hour,day_name,period_of_day
0,11.0,2021-05-18 21:34:00,18.0,21.0,Tuesday,night
1,9.1,2021-05-18 15:36:00,18.0,15.0,Tuesday,midday
2,14.0,2021-05-18 05:55:00,18.0,5.0,Tuesday,dawn
3,19.0,2021-05-18 04:49:00,18.0,4.0,Tuesday,dawn
4,22.0,2021-05-18 05:22:00,18.0,5.0,Tuesday,dawn


In [5]:
grouped = df_all.groupby(['hour','day_name','day','period_of_day'])['amount'].count().reset_index()

In [6]:
grouped.rename({'amount':'transactions'}, axis=1, inplace=True)

In [7]:
grouped = grouped[grouped['day'] != 17]

In [8]:
grouped.head()

,hour,day_name,day,period_of_day,transactions
0,0.0,Friday,21.0,midnight,18
1,0.0,Friday,28.0,midnight,12
2,0.0,Monday,24.0,midnight,29
3,0.0,Monday,31.0,midnight,10
4,0.0,Saturday,22.0,midnight,100


In [9]:
days = grouped['day'].value_counts().head(20).reset_index()

In [10]:
days['index']=days['index'].astype(int)

In [11]:
days.sort_values(by='index', inplace=True)

In [12]:
days.head()

,index,day
3,18,24
6,19,24
2,20,24
8,21,24
9,22,24


### Creating the Dashboard

In [13]:
def value_filter(df, value):
    df = df[(df['day']== float(value))]
    return df

In [14]:
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

app.layout = dbc.Container([
 
#FIRST ROW----------------------(HEADER)-------------------------------------------------------  
    dbc.Row([ 
       dbc.Col(html.H1('Taxi Service Usage Report', 
                        className = 'text-center text-primary mb-4'),
                width=12)
    ]),
                    
#SECOND ROW------------------(DROPDOWN)-------------------------------------------------------- 
    dbc.Row([
        dbc.Col([html.Label("Select Date:",
                   style={'font-weight': 'bold',"textDecoration": "underline"}, className = 'text-center mb4'),
            
            dcc.Dropdown(id='my-dropdown', value= 18, searchable=False,
                              options = [{'label':str(x)+' May', 'value':x}
                                        for x in days['index']],
                          style={'width':"100%"}),
                 
                  html.Div(id='slider-output-container'),
                
                     html.Br(),
        
                ],width={'size':6,'offset':3}, className='mb-4'),
        
    ]),
    
#THIRD ROW------------------(BAR GRAPHS)---------------------------------------------------- 
    dbc.Row([

     
    #first column ---------------------------------------------------       
        dbc.Col([dcc.Graph(id="barplot", figure={}),               #first barplot, period of the day
             ]),
    
            html.Br(),
    
    #second column--------------------------------------------------       
            dbc.Col([dcc.Graph(id="barplot2", figure={}) ,              #second barplot, hours
            html.Br()
                    ])
        ]),
    
#FOURTH ROW------------------(LINEPLOT)------------------------------------ 
            
    dbc.Row([

        dbc.Col([dcc.Graph(id="lineplot", figure={}),               #first barplot, crimes
             ]),
    
            html.Br(),
        ]),
     ])
#callback for barplots----------------------------------------------------------------------

@app.callback(
    [Output('barplot', component_property ='figure'),
     Output('barplot2', component_property ='figure'),
    Output('slider-output-container', component_property ='children')],
    Input('my-dropdown', component_property ='value')
)
def update_graph(value):
    dff=df_all
    day = value_filter(dff,value)
    fig = px.histogram(day, x='period_of_day', y= 'amount',histfunc= 'avg',
                        height=500, 
                   labels={'period_of_day':'Period of The Day','amount': 'Amount Paid'},
                       title ='Average amount paid by period of the day')
    
    fig2 = px.histogram(day, x='hour', y= 'amount',histfunc= 'avg',
                        height=500, color_discrete_sequence=['indianred'],
                   labels={'hour':'Hour of The Day','amount': 'Amount Paid'},
                       title ='Average amount paid by the hour')
    
    return fig, fig2, '{} May 2021'.format(day['day_name'].unique()[0])
    

#callback for lineplot----------------------------------------------------------------------
    
@app.callback(
    Output("lineplot", component_property ='figure'),
     [Input('my-dropdown', component_property ='value')]
     #Input('range-slider', component_property ='value')]
)
def update_graph(value):
    dff = grouped
    fig = px.line(value_filter(dff,value), x='hour', y='transactions',color='day_name',
                  labels={'hour':'Hours of The Day','transactions': 'Number of Transactions'},
                 title ='Taxi Usage by Hour')
    
    fig.update_layout(legend=dict(
    title=None,
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1))
    
    return fig 
    
    
    
app.run_server(mode ='external',host="localhost", port=8051)

Dash app running on http://localhost:8051/
